In [21]:
class Operation():
    
    def __init__(self,input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes =[]
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
    
    def compute(self):
        pass 

In [22]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        return x_var + y_var

In [23]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        return x_var * y_var

In [24]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        
        return x_var.dot(y_var)

In [25]:
class Placeholder():
        
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [26]:
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [27]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self

In [28]:
g = Graph()

In [29]:
g.set_as_default()

In [30]:
A = Variable(10)

In [31]:
b = Variable(1)

In [32]:
x = Placeholder()

In [33]:
y = multiply(A,x)

In [34]:
z = add(y,b)

In [35]:
import numpy as np

In [36]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [40]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output

In [41]:
sess = Session()

In [42]:
result = sess.run(operation=z,feed_dict={x:10})

g = Graph()

g.set_as_default()

A = Variable([10,20], [30,40])
b = Variable([1,2])

x = Placeholder()

y = matmul(A,x)

z = add(y,b)